In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score
df=pd.read_csv("training_set.csv")
df['Loan_ID'] = df['Loan_ID'].str.replace("LP", "", regex=True).astype(int)
df_encoded = pd.get_dummies(df['property_Area'], prefix='property_Area')
df = pd.concat([df, df_encoded], axis=1)
df.rename(columns={'property_Area_Urban': 'Urban', 'property_Area_Rural': 'Rural', 'property_Area_Semiurban': 'Semiurban'}, inplace=True)
df_encoded_gender = pd.get_dummies(df['Gender'], prefix='Gender')
df = pd.concat([df, df_encoded_gender], axis=1)
df.rename(columns={'Gender_Male': 'Male', 'Gender_Female': 'Female'}, inplace=True)
categorical_columns = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
                       'property_Area', 'Loan_Status','Rural','Urban','Semiurban','Female','Male']

# Convert to numerical
for col in categorical_columns:
    df[col] = df[col].astype('category').cat.codes
df.replace(-1, np.nan, inplace=True)

for col in categorical_columns:
    df[col].fillna(df[col].mode()[0], inplace=True)
df.isna().sum()

num_cols = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term']
for col in num_cols:
    df[col].fillna(df[col].median(), inplace=True)

df['Credit_History'].fillna(df['Credit_History'].mode()[0], inplace=True)

df['Total_Income'] = df['ApplicantIncome'] + df['CoapplicantIncome']
df['Loan_Income_Ratio'] = df['LoanAmount'] / df['Total_Income']
df['EMI'] = df['LoanAmount'] / df['Loan_Amount_Term']
df['Balance_Income'] = df['Total_Income'] - df['EMI']

numerical_features = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Total_Income', 'Loan_Income_Ratio', 'EMI', 'Balance_Income']


from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler
vif_features = df[numerical_features].dropna()

# Standardize features for VIF
scaler = StandardScaler()
vif_scaled = scaler.fit_transform(vif_features)

# Calculating VIF
vif_data = pd.DataFrame()
vif_data["Feature"] = vif_features.columns
vif_data["VIF"] = [variance_inflation_factor(vif_scaled, i) for i in range(vif_scaled.shape[1])]


df.drop(columns=['Total_Income', 'CoapplicantIncome', 'Balance_Income', 'EMI'], inplace=True)
df.drop(columns=['property_Area', 'Gender'], inplace=True)
corr2 = df.corr()

target_corr2 = corr2['Loan_Status'].dropna().sort_values(ascending=False)
target_corr2 = target_corr2[abs(target_corr2) > 0.03].sort_values(key=abs, ascending=False)

selected_features = [
    'Credit_History', 'Semiurban', 'Rural', 'Married',
    'Education', 'Loan_Income_Ratio', 'Urban', 'LoanAmount'
]

x = df[selected_features]
y = df['Loan_Status']

x
y

from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

x_train
x_val
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Scale the features
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_val_scaled = scaler.transform(x_val)

# KNN classifier
knn = KNeighborsClassifier(n_neighbors=11)  # You can tune n_neighbors
knn.fit(x_train_scaled, y_train)

# Predictions
y_pred = knn.predict(x_val_scaled)

# Evaluation
print("Accuracy:", accuracy_score(y_val, y_pred))
print("\nClassification Report:\n", classification_report(y_val, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_val, y_pred))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

dt = pd.read_csv("testing_set.csv")

dt['Loan_ID'] = dt['Loan_ID'].str.replace("LP", "", regex=True).astype(int)

dt_encoded = pd.get_dummies(dt['property_Area'], prefix='property_Area')
dt = pd.concat([dt, dt_encoded], axis=1)
dt.rename(columns={'property_Area_Urban': 'Urban', 'property_Area_Rural': 'Rural', 'property_Area_Semiurban': 'Semiurban'}, inplace=True)

dt_encoded_gender = pd.get_dummies(dt['Gender'], prefix='Gender')
dt = pd.concat([dt, dt_encoded_gender], axis=1)
dt.rename(columns={'Gender_Male': 'Male', 'Gender_Female': 'Female'}, inplace=True)

categorical_columns = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
                       'property_Area','Rural','Urban','Semiurban','Female','Male']

# Convert to numerical
for col in categorical_columns:
    dt[col] = dt[col].astype('category').cat.codes
dt.replace(-1, np.nan, inplace=True)

for col in categorical_columns:
    dt[col].fillna(dt[col].mode()[0], inplace=True)

dt.isna().sum()

num_cols = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term']
for col in num_cols:
    dt[col].fillna(dt[col].median(), inplace=True)

dt['Credit_History'].fillna(dt['Credit_History'].mode()[0], inplace=True)

dt['Total_Income'] = dt['ApplicantIncome'] + dt['CoapplicantIncome']
dt['Loan_Income_Ratio'] = dt['LoanAmount'] / dt['Total_Income']
dt['EMI'] = dt['LoanAmount'] / dt['Loan_Amount_Term']
dt['Balance_Income'] = dt['Total_Income'] - dt['EMI']

numerical_features = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Total_Income', 'Loan_Income_Ratio', 'EMI', 'Balance_Income']

from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler

vif_features = dt[numerical_features].dropna()

# Standardize features for VIF
scaler = StandardScaler()
vif_scaled = scaler.fit_transform(vif_features)

# Calculating VIF
vif_data = pd.DataFrame()
vif_data["Feature"] = vif_features.columns
vif_data["VIF"] = [variance_inflation_factor(vif_scaled, i) for i in range(vif_scaled.shape[1])]

dt.drop(columns=['Total_Income', 'CoapplicantIncome', 'Balance_Income', 'EMI'], inplace=True)
dt.drop(columns=['property_Area', 'Gender'], inplace=True)



selected_features = [
    'Credit_History', 'Semiurban', 'Rural', 'Married',
    'Education', 'Loan_Income_Ratio', 'Urban', 'LoanAmount'
]

x_test = dt[selected_features]


x_test


scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)

# Train KNN
knn = KNeighborsClassifier(n_neighbors=11)
knn.fit(x_train_scaled, y_train)

# Scale test set using the same scaler
x_test_scaled = scaler.transform(x_test)

# Predict
y_pred_test = knn.predict(x_test_scaled)

y_pred_test


Accuracy: 0.7886178861788617

Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.42      0.58        43
           1       0.76      0.99      0.86        80

    accuracy                           0.79       123
   macro avg       0.85      0.70      0.72       123
weighted avg       0.83      0.79      0.76       123


Confusion Matrix:
 [[18 25]
 [ 1 79]]


C:\Users\Richa\AppData\Local\Temp\ipykernel_22580\2916210618.py:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)
C:\Users\Richa\AppData\Local\Temp\ipykernel_22580\2916210618.py:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For ex

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,

In [61]:
df_full_encoded=df.copy()

In [62]:
# Filter ineligible customers
ineligible_customers = df_full_encoded[df['Loan_Status'] == 0]

# Features: All except target
X_amount = ineligible_customers.drop(['LoanAmount'], axis=1)
y_amount = ineligible_customers['LoanAmount']

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X_amount, y_amount, test_size=0.2, random_state=42)


from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
# Scale
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_val_scaled = scaler.transform(X_val)

# Model
reg_amount = LinearRegression()
reg_amount.fit(X_train, y_train)

# Predict & Evaluate
y_pred_amount = reg_amount.predict(X_val)

mae_amount = mean_absolute_error(y_val, y_pred_amount)
rmse_amount = np.sqrt(mean_squared_error(y_val, y_pred_amount))

print("Task 2 - Loan Amount Prediction:")
print("MAE:", mae_amount)
print("RMSE:", rmse_amount)


Task 2 - Loan Amount Prediction:
MAE: 52.81928808061028
RMSE: 82.4423615141831


In [63]:
y_pred_amount

array([144.68989923, 154.83060411, 193.95470534, 178.05027659,
       156.0654287 , 139.74882097, 146.77255278, 147.14815626,
       140.53451732, 180.72238105, 152.09984029, 110.91055012,
       166.19477418, 182.45814437, 125.0791467 , 138.10943349,
       120.91278224, 163.57748574, 137.54236827, 166.24213661,
       115.56051204, 117.61754436, 171.62125304, 211.6809453 ,
       171.96124979, 225.11238718, 131.49331577, 137.11758245,
       107.81411667, 135.33411006, 121.67714342, 106.1956315 ,
       189.95709046, 144.66460445, 167.86277147, 140.21379113,
       145.24616048, 156.06186684, 177.53162768])

In [64]:
X_train.head()

,Loan_ID,Married,Dependents,Education,Self_Employed,ApplicantIncome,Loan_Amount_Term,Credit_History,Loan_Status,Rural,Semiurban,Urban,Female,Male,Loan_Income_Ratio
256,1849,0.0,0.0,1.0,0.0,6045.0,360.0,0.0,0,1,0,0,0,1,0.019024
357,2151,1.0,1.0,0.0,0.0,3875.0,360.0,1.0,0,0,0,1,0,1,0.017290
519,2684,0.0,0.0,1.0,0.0,3400.0,360.0,1.0,0,1,0,0,1,0,0.027941
95,1326,0.0,0.0,0.0,0.0,6782.0,360.0,1.0,0,0,0,1,0,1,0.018873
431,2379,0.0,0.0,0.0,0.0,6500.0,360.0,0.0,0,1,0,0,0,1,0.016154


In [65]:
test=dt.copy()
test['Loan_Status'] = y_pred_test
test

,Loan_ID,Married,Dependents,Education,Self_Employed,ApplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Rural,Semiurban,Urban,Female,Male,Loan_Income_Ratio,Loan_Status
0,1015,1,0.0,0,0.0,5720,110.0,360.0,1.0,0,0,1,0,1,0.019231,1
1,1022,1,1.0,0,0.0,3076,126.0,360.0,1.0,0,0,1,0,1,0.027535,1
2,1031,1,2.0,0,0.0,5000,208.0,360.0,1.0,0,0,1,0,1,0.030588,1
3,1035,1,2.0,0,0.0,2340,100.0,360.0,1.0,0,0,1,0,1,0.020467,1
4,1051,0,0.0,1,0.0,3276,78.0,360.0,1.0,0,0,1,0,1,0.023810,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,2971,1,3.0,1,1.0,4009,113.0,360.0,1.0,0,0,1,0,1,0.019530,1
363,2975,1,0.0,0,0.0,4158,115.0,360.0,1.0,0,0,1,0,1,0.023629,1
364,2980,0,0.0,0,0.0,3250,126.0,360.0,1.0,0,1,0,0,1,0.024032,1
365,2986,1,0.0,0,0.0,5000,158.0,360.0,1.0,1,0,0,0,1,0.021372,1


In [66]:
# Select only ineligible customers (Loan_Status == 0) in test set
ineligible_test = test[test['Loan_Status'] == 0]

# Drop target column to get features only
test_x = ineligible_test.drop(['LoanAmount'], axis=1)

# OPTIONAL: If you have true LoanAmount in test set for evaluation
test_y = ineligible_test['LoanAmount']

# Reorder columns to match training
test_x = test_x.reindex(columns=X_train.columns, fill_value=0)


In [67]:
test_x.head()

,Loan_ID,Married,Dependents,Education,Self_Employed,ApplicantIncome,Loan_Amount_Term,Credit_History,Loan_Status,Rural,Semiurban,Urban,Female,Male,Loan_Income_Ratio
7,1056,1,2.0,1,0.0,3881,360.0,0.0,0,1,0,0,0,1,0.037877
13,1094,1,2.0,0,0.0,12173,360.0,0.0,0,0,1,0,0,1,0.013637
25,1153,0,0.0,0,0.0,0,360.0,0.0,0,1,0,0,0,1,0.006167
35,1203,0,0.0,0,0.0,3150,360.0,0.0,0,0,1,0,0,1,0.055873
55,1313,0,0.0,0,0.0,2750,360.0,0.0,0,0,0,1,0,1,0.047273


In [68]:
X_train

,Loan_ID,Married,Dependents,Education,Self_Employed,ApplicantIncome,Loan_Amount_Term,Credit_History,Loan_Status,Rural,Semiurban,Urban,Female,Male,Loan_Income_Ratio
256,1849,0.0,0.0,1.0,0.0,6045.0,360.0,0.0,0,1,0,0,0,1,0.019024
357,2151,1.0,1.0,0.0,0.0,3875.0,360.0,1.0,0,0,0,1,0,1,0.017290
519,2684,0.0,0.0,1.0,0.0,3400.0,360.0,1.0,0,1,0,0,1,0,0.027941
95,1326,0.0,0.0,0.0,0.0,6782.0,360.0,1.0,0,0,0,1,0,1,0.018873
431,2379,0.0,0.0,0.0,0.0,6500.0,360.0,0.0,0,1,0,0,0,1,0.016154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,2130,1.0,0.0,1.0,0.0,3523.0,360.0,0.0,0,1,0,0,0,1,0.022509
34,1100,0.0,3.0,0.0,0.0,12500.0,360.0,1.0,0,1,0,0,0,1,0.020645
293,1945,0.0,0.0,0.0,0.0,5417.0,480.0,0.0,0,0,0,1,1,0,0.026398
574,2863,1.0,3.0,0.0,0.0,6406.0,360.0,1.0,0,0,1,0,0,1,0.023416


In [69]:
reg_amount = LinearRegression()
reg_amount.fit(X_train, y_train)

# Predict & Evaluate
y_pred_amount_test = reg_amount.predict(test_x)

mae_amount = mean_absolute_error(test_y, y_pred_amount_test)
rmse_amount = np.sqrt(mean_squared_error(test_y, y_pred_amount_test))
r2 = r2_score(test_y, y_pred_amount_test)

print("Task 2 - Loan Amount Prediction:")
print("MAE:", mae_amount)
print("RMSE:", rmse_amount)
print("R2:", r2)


Task 2 - Loan Amount Prediction:
MAE: 30.899058038403325
RMSE: 41.05695799038582
R2: 0.009058497220887918


In [70]:
y_pred_amount_test

array([150.10963212, 200.52162707,  81.15008314, 177.46061504,
       151.93071023, 159.19225526, 108.83691166, 136.75632142,
       171.83942323, 135.08379394, 181.55248575, 142.15288381,
       170.04792062, 157.94646343, 171.8284815 , 109.9406257 ,
       196.4436658 , 128.08869777,  99.82199142, 176.18691057,
       109.07283387, 151.61573044, 118.11698032, 133.16201296,
       144.84012436, 125.44039665, 113.46642168, 133.82635529,
       164.16044067, 114.96489694, 148.06368709, 126.04969699,
       102.23205318, 126.69196628, 130.73408402, 162.22327816,
       160.03203399, 125.08005449, 170.80319353, 140.05044344,
       125.07318546,  97.14534757, 141.41181785, 133.75902082,
       130.46938655, 174.92014843, 134.50624682,  94.97070039,
       136.25873825, 168.40635142, 116.73712478, 161.3302514 ,
       109.07408775, 148.48524441, 111.26744801,  80.74222758,
       122.70633367,  99.72443087, 127.31627002, 158.25146004])

In [71]:
# Filter customers who are ineligible AND have requested <= 20 years duration (i.e., <= 240 months)
ineligible_duration_customers = df_full_encoded[
    (df['Loan_Status'] == 0) & (df['Loan_Amount_Term'] <= 240)
]

# Target: Loan_Amount_Term (predict the minimum duration needed)
y_duration = ineligible_duration_customers['Loan_Amount_Term']

# Features: All except 'Loan_Amount_Term' (and ideally, also 'Loan_Status' since it’s your label)
X_duration = ineligible_duration_customers.drop(['Loan_Amount_Term'], axis=1)

# Train-test split
X_train_dur, X_val_dur, y_train_dur, y_val_dur = train_test_split(X_duration, y_duration, test_size=0.2, random_state=42)

# Fit regression model
reg_duration = LinearRegression()
reg_duration.fit(X_train_dur, y_train_dur)

# Predict
y_pred_dur = reg_duration.predict(X_val_dur)

# Evaluate
mae_dur = mean_absolute_error(y_val_dur, y_pred_dur)
rmse_dur = np.sqrt(mean_squared_error(y_val_dur, y_pred_dur))

print("\nTask 3 - Loan Duration Adjustment:")
print("MAE:", mae_dur)
print("RMSE:", rmse_dur)



Task 3 - Loan Duration Adjustment:
MAE: 190.28943126928937
RMSE: 245.83053387914853


In [72]:
ineligible_duration_customers.head()

,Loan_ID,Married,Dependents,Education,Self_Employed,ApplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Rural,Semiurban,Urban,Female,Male,Loan_Income_Ratio
62,1207,1.0,0.0,1.0,1.0,2609.0,165.0,180.0,0.0,0,1,0,0,0,1,0.027237
66,1228,0.0,0.0,1.0,0.0,3200.0,126.0,180.0,0.0,0,0,0,1,0,1,0.023102
128,1451,1.0,1.0,0.0,1.0,10513.0,160.0,180.0,0.0,0,0,0,1,0,1,0.011140
172,1586,1.0,3.0,1.0,0.0,3522.0,81.0,180.0,1.0,0,1,0,0,0,1,0.022998
202,1682,1.0,3.0,1.0,0.0,3992.0,128.0,180.0,1.0,0,0,0,1,0,1,0.032064


In [73]:
X_duration.shape

(19, 15)

In [74]:
y_pred_dur

array([637.38001463, 291.66232359, 247.67202521, 304.44336164])

In [75]:
X_val_dur

,Loan_ID,Married,Dependents,Education,Self_Employed,ApplicantIncome,LoanAmount,Credit_History,Loan_Status,Rural,Semiurban,Urban,Female,Male,Loan_Income_Ratio
62,1207,1.0,0.0,1.0,1.0,2609.0,165.0,0.0,0,1,0,0,0,1,0.027237
241,1800,1.0,1.0,1.0,0.0,2510.0,140.0,1.0,0,0,0,1,0,1,0.031160
471,2517,1.0,1.0,1.0,0.0,2653.0,113.0,0.0,0,1,0,0,0,1,0.027209
66,1228,0.0,0.0,1.0,0.0,3200.0,126.0,0.0,0,0,0,1,0,1,0.023102


In [76]:
# Filter test set with same condition: ineligible + requested duration <= 20 years
ineligible_duration_test = test[
    (test['Loan_Status'] == 0) & (test['Loan_Amount_Term'] <= 240)
]

test_x_dur = ineligible_duration_test.drop(['Loan_Amount_Term'], axis=1)
test_y_dur = ineligible_duration_test['Loan_Amount_Term']

# Match column order
test_x_dur = test_x_dur.reindex(columns=X_train_dur.columns, fill_value=0)

# Predict and evaluate
y_pred_dur_test = reg_duration.predict(test_x_dur)
mae_dur_test = mean_absolute_error(test_y_dur, y_pred_dur_test)
rmse_dur_test = np.sqrt(mean_squared_error(test_y_dur, y_pred_dur_test))
r2_dur_test = r2_score(test_y_dur, y_pred_dur_test)

print("\nTask 3 (Test) - Loan Duration Adjustment:")
print("MAE:", mae_dur_test)
print("RMSE:", rmse_dur_test)
print("R² Score:", r2_dur_test)



Task 3 (Test) - Loan Duration Adjustment:
MAE: 95.33259827606699
RMSE: 102.59136667904505
R² Score: -1.1727169451616537


In [77]:
y_pred_dur_test

array([327.31360354, 256.12874248, 234.5890404 , 140.1068846 ,
       115.47527963])

In [78]:
test_x_dur

,Loan_ID,Married,Dependents,Education,Self_Employed,ApplicantIncome,LoanAmount,Credit_History,Loan_Status,Rural,Semiurban,Urban,Female,Male,Loan_Income_Ratio
84,1450,1,0.0,0,0.0,4456,131.0,0.0,0,0,1,0,0,1,0.029399
173,1979,0,0.0,0,0.0,3017,159.0,0.0,0,0,0,1,0,1,0.027124
245,2355,1,0.0,0,0.0,3186,150.0,0.0,0,0,1,0,0,0,0.023693
325,2802,0,0.0,0,0.0,2875,95.0,0.0,0,0,1,0,0,1,0.017955
354,2921,1,3.0,1,0.0,5316,158.0,0.0,0,0,1,0,0,1,0.028712
